In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, activations, models, preprocessing, utils
import re
import os
import yaml

### Criar uma lista com todos os arquivos

In [2]:
dir_path = '../../../dados/portuguese/'
files_list = os.listdir(dir_path + os.sep)

### Carregar os arquivos em dois vetores questions e answers

In [3]:
questions = []
answers = []

for filepath in files_list:
    file = open(dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(file)
    conversations = docs['conversations']
    for con in conversations:
        if len(con) > 1 :
            questions.append(con[0])
            answers.append(con[1])

### Colocar as tag de START e END nas respostas

In [77]:
len(questions), len(answers)

(280, 280)

In [4]:
answers_tags = []
for i in range(len(answers)) :
    answers_tags.append('<START> ' + answers[i] + ' <END>')


### Tokenizar e criar o vocabulario

In [5]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers_tags)
VOCAB_SIZE = len(tokenizer.word_index)+1

In [30]:
tokenizer.word_index

{'start': 1,
 'end': 2,
 'o': 3,
 'é': 4,
 'de': 5,
 'que': 6,
 'você': 7,
 'e': 8,
 'a': 9,
 'um': 10,
 'do': 11,
 'como': 12,
 'da': 13,
 'em': 14,
 'não': 15,
 'para': 16,
 'bem': 17,
 'quem': 18,
 'uma': 19,
 'os': 20,
 'muito': 21,
 'eu': 22,
 'tem': 23,
 'por': 24,
 'mais': 25,
 'vai': 26,
 'qual': 27,
 'foi': 28,
 'na': 29,
 'bom': 30,
 'se': 31,
 'com': 32,
 'me': 33,
 'ações': 34,
 'ou': 35,
 'mercado': 36,
 'também': 37,
 'seu': 38,
 'está': 39,
 'estou': 40,
 'tudo': 41,
 'são': 42,
 'obrigado': 43,
 'tu': 44,
 'minha': 45,
 'as': 46,
 'no': 47,
 'aí': 48,
 'dinheiro': 49,
 'mas': 50,
 'ser': 51,
 'gosto': 52,
 'melhor': 53,
 'acha': 54,
 'nome': 55,
 'quais': 56,
 'quanto': 57,
 'linguística': 58,
 'faço': 59,
 'ele': 60,
 'ela': 61,
 'sim': 62,
 'eae': 63,
 'unilab': 64,
 'obrigada': 65,
 'das': 66,
 'és': 67,
 'vida': 68,
 'sou': 69,
 'isso': 70,
 'fala': 71,
 'te': 72,
 'beleza': 73,
 'dia': 74,
 'quer': 75,
 'comprar': 76,
 'ai': 77,
 'até': 78,
 'letras': 79,
 'ter': 8

In [7]:
VOCAB_SIZE

1220

### Transformar as questões em sequencias de numeros

In [31]:
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
encoder_input_data = preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=maxlen_questions, padding='post')

In [32]:
questions[1:5]

['adoro você',
 'gosto da sua sinceridade',
 'você é valente',
 'você é muito sexy']

In [47]:
maxlen_questions

28

In [33]:
tokenized_questions[1:5]

[[383, 7], [52, 13, 83, 199], [7, 4, 384], [7, 4, 21, 385]]

In [35]:
encoder_input_data[1:5]

array([[383,   7,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 52,  13,  83, 199,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  7,   4, 384,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  7,   4,  21, 385,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])

In [9]:
print(encoder_input_data.shape)

(280, 28)


### Transformar as respostas em sequencias de numeros

In [37]:
tokenized_answers = tokenizer.texts_to_sequences(answers_tags)

In [38]:
maxlen_answers = max([len(x) for x in tokenized_answers])
decoder_input_data = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')

In [41]:
maxlen_answers

47

In [40]:
tokenized_answers[1:5]

[[1, 37, 2], [1, 6, 30, 2], [1, 43, 2], [1, 7, 4, 6, 4, 2]]

In [39]:
print(decoder_input_data.shape)

(280, 47)


### Fazer os as respostas para o decodificador de saida sem a tag START

In [13]:
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_output_data = utils.to_categorical(padded_answers , VOCAB_SIZE)

In [36]:
tokenized_answers[1:5]

[[37, 2], [6, 30, 2], [43, 2], [7, 4, 6, 4, 2]]

In [46]:
decoder_output_data[1:5]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]], dtyp

In [15]:
print(decoder_output_data.shape)

(280, 47, 1220)


### Fazer o encoder de entrada

In [16]:
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_questions ,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(200 , return_state=True)(encoder_embedding)
encoder_states = [ state_h , state_c ]

### Fazer o decoder de entrada

In [17]:
decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers , ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)

### Fazer o encoder de saida

In [ ]:
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense (decoder_outputs)

### Construir o modelo

In [18]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 47)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 28, 200)              244000    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 47, 200)              244000    ['input_2[0][0]']             
                                                                                              

### Treinar o modelo

In [20]:
model.fit([encoder_input_data, decoder_input_data], decoder_output_data, batch_size=32, epochs=200) 

Epoch 1/200
9/9 [==============================] - 15s 186ms/step - loss: 7.0837
Epoch 2/200
9/9 [==============================] - 2s 211ms/step - loss: 6.6899
Epoch 3/200
9/9 [==============================] - 2s 184ms/step - loss: 5.7352
Epoch 4/200
9/9 [==============================] - 2s 196ms/step - loss: 5.4824
Epoch 5/200
9/9 [==============================] - 2s 194ms/step - loss: 5.3935
Epoch 6/200
9/9 [==============================] - 2s 205ms/step - loss: 5.3224
Epoch 7/200
9/9 [==============================] - 2s 210ms/step - loss: 5.2680
Epoch 8/200
9/9 [==============================] - 2s 194ms/step - loss: 5.2420
Epoch 9/200
9/9 [==============================] - 2s 222ms/step - loss: 5.1929
Epoch 10/200
9/9 [==============================] - 3s 305ms/step - loss: 5.1383
Epoch 11/200
9/9 [==============================] - 2s 235ms/step - loss: 5.0739
Epoch 12/200
9/9 [==============================] - 2s 201ms/step - loss: 5.0107
Epoch 13/200
9/9 [==================

In [21]:
encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
decoder_state_input_h = tf.keras.layers.Input(shape=(200 ,))
decoder_state_input_c = tf.keras.layers.Input(shape=(200 ,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding , initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.models.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [22]:
def preprocess(input_sentence):
    tokens = input_sentence.lower().split()
    tokens_list = []
    for word in tokens:
        tokens_list.append(tokenizer.word_index[word]) 
    return preprocessing.sequence.pad_sequences([tokens_list] , maxlen=maxlen_questions , padding='post')

In [74]:
tokenizer.word_index['batida']

KeyError: 'batida'

In [75]:
tests = ['oi', 'quem é você', 'qual seu livro favorito', 'você é um humano ou computador', 'o que é linguística', 'o que é uma opinião', 'até mais']

In [76]:
for i, frase in enumerate(tests):
    frase_processada = preprocess(tests[i])
    states_values = encoder_model.predict(frase_processada)
    empty_target_seq = np.zeros((1 , 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    
    while not stop_condition :
        dec_outputs , h , c = decoder_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += f' {word}'
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros((1 , 1))  
        empty_target_seq[0 , 0] = sampled_word_index
        states_values = [h , c] 
    print(f'Human: {tests[i]}')
    print()
    
    decoded_translation = decoded_translation.split(' end')[0]
    print(f'Bot: {decoded_translation}')
    print('-'*25)

1/1 [==============================] - 0s 24ms/step
Human: oi

Bot:  olá
-------------------------
1/1 [==============================] - 0s 52ms/step
Human: quem é você

Bot:  eu estou bem e você
-------------------------
1/1 [==============================] - 0s 49ms/step
Human: qual seu livro favorito

Bot:  não sei
-------------------------
1/1 [==============================] - 0s 29ms/step
Human: você é um humano ou computador

Bot:  que bom
-------------------------
1/1 [==============================] - 0s 28ms/step
Human: o que é linguística

Bot:  tecnicamente é o estudo é a ciência que estuda os fenômenos relacionados de texto linguística de escassez
-------------------------
1/1 [==============================] - 0s 30ms/step
Human: o que é uma opinião

Bot:  bem a alfabetização pode dar a parole como individual e uma escrita em um nome é um palavra que denomina o nome é o nome é o nome é um palavra que denomina o nome
-------------------------
1/1 [========================